<a href="https://colab.research.google.com/github/ngeklai/BADS7105-CRM-Analysis-and-Intelligence/blob/main/Doc_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Prepare Data**

In [2]:
#grouping comments for each restaurant to be one long text

import pandas as pd

documents = pd.read_csv('/content/drive/MyDrive/CRM Analytics/11. Voices of Customers/CustomerReviews(R01).csv')
documents['Restaurant']

0    Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ
1            Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G
2                                        ข้าน้อยขอชาบู
Name: Restaurant, dtype: object

# **2. Install & Import**

In [3]:
!pip install --upgrade pythainlp
import pythainlp
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.feature_extraction.text import TfidfVectorizer 
import numpy as np 

     |████████████████████████████████| 11.0MB 17.8MB/s 
     |████████████████████████████████| 747kB 38.6MB/s 


# **3. Text Preprocessing**

Source: from class BADS7105

In [4]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')' , '           ']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [5]:
documents['Review_tokenized'] = documents['Review'].apply(lambda x: tokenize_with_space(x))

# **4. TF-IDF and Vector Conversion**

Source: https://towardsdatascience.com/calculating-document-similarities-using-bert-and-other-models-b2c1a29c9630

In [6]:
tfidfvectoriser=TfidfVectorizer()
tfidfvectoriser.fit(documents['Review_tokenized'] )

tfidf_vectors=tfidfvectoriser.transform(documents['Review_tokenized'] )

# **5. Cosine Similarity**

In [7]:
pairwise_similarities=np.dot(tfidf_vectors,tfidf_vectors.T).toarray()
pairwise_similarities

array([[1.        , 0.1488546 , 0.09067434],
       [0.1488546 , 1.        , 0.09143388],
       [0.09067434, 0.09143388, 1.        ]])

In [30]:
print (tfidf_vectors[0].toarray())
print (pairwise_similarities.shape)
print (pairwise_similarities[0][:])

[[0.         0.         0.         0.         0.         0.10274067
  0.         0.         0.10274067 0.         0.308222   0.
  0.         0.         0.         0.10274067 0.10274067 0.
  0.10274067 0.         0.         0.10274067 0.         0.
  0.10274067 0.         0.         0.         0.10274067 0.10274067
  0.         0.         0.         0.10274067 0.         0.
  0.         0.308222   0.06068029 0.20548134 0.20548134 0.10274067
  0.10274067 0.10274067 0.         0.         0.10274067 0.
  0.07813693 0.         0.23441078 0.         0.         0.06068029
  0.         0.         0.         0.         0.10274067 0.10274067
  0.         0.         0.         0.         0.         0.10274067
  0.         0.10274067 0.10274067 0.308222   0.         0.308222
  0.         0.23441078 0.         0.10274067 0.07813693 0.
  0.         0.         0.10274067 0.10274067 0.10274067 0.
  0.         0.         0.10274067 0.07813693 0.         0.10274067
  0.         0.15627385 0.         0.1

In [9]:
#identify most similarity

def most_similar(doc_id,similarity_matrix,matrix):
    print (f'Document: {documents.iloc[doc_id]["Review"]}')
    print ('\n')
    print ('Similar Documents:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    for ix in similar_ix:
        if ix==doc_id:
            continue
        print('\n')
        print (f'Document: {documents.iloc[ix]["Review"]}')
        print (f'{matrix} : {similarity_matrix[doc_id][ix]}')

In [31]:
most_similar(1,pairwise_similarities,'Cosine Similarity')

#index 0: Mo-Mo-Paradise [ที่สำคัญของร้านนี้...]
#index 1: Shabushi [มา านที่ขาบูชิต้องมาตอนหิว...]
#index 2: ข้าน้อยขอชาบู [หลังจากที่เคยลองสาขา...]

Document: มา านที่ขาบูชิต้องมาตอนหิว ไม่งั้นจะไม่คุ้มนะครับ ฮ่าๆ เพราะเมนูของกินมันเยอะมาก ทั้งอาหารบนสายพาน อาหารทานเล่น ซูชิ ปูอัด เกี๊ยวซ่า กุ้งเทมปุระ มันละลานตามาก ราคาตอนนี้ 399 บาท แพงมั้ย..ก็นิดนึง แต่ถ้าเทียบกับอาหารแล้วก็ถือว่าคุ้มครับ


Similar Documents:


Document: ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักสุดๆสะอาดสะอ้านใส่ใจค่อนข้างประทับใจกับการบริการรสชาติอาหารอันนี้แปลก​ใหม่
เซ็ท560 บาท
อาหารจำกัด​ไม่ได้​หลากหลาย​แต่เพียง​พอกับความต้องการ
เราไม่กินเนื้อ​ กินสันคอ4 ถาด​ เป็ด1
แฟนกินเนื้อ​ คุณ​ภาพ​ดี
น้ำซุป​ดำ​กับเผ็ด
ที่สำคัญคือน้ำจิ้มอร่อยมากไม่เคยกินรสชาติแบบนี้มาก่อนเป็นเต้าเจี้ยวนิดๆแต่ก็ไม่ได้เคลมก็รสชาติดีนะถามราคาโดยรวมก็ถือว่าคุ้มเพราะว่าโมจิอร่อยมากไอติมก็อร่อยไม่ต้องออกไปกินข้างนอกสรุปรวมๆละกันว่าประทับใจในบริการและคุณภาพของอาหาร
Cosine Similarity : 0.14885459828163455


Document: หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที่สาขาเดอะมอลล์บางกะปิชั้น 4 โซน MCC HALL ด้านในสุดติดกับพวกค่ายมวย ตอนแรกก็นึกว่าชาบู 199 ธรรมดา แต่ที่ไหนได้ มีพิซซ่าจากนารายพิซเซอ

As the result, we found that...
1. Momo Paradise: Shabushi is its competitor
2. Shabushi: Momo Paradise is its competitor
3. Khanoi: Shabushi is its competitor

Alternative methods: Word2Vec, GloVe, FastText, BERT (and other Doc2Vec tools)